In [1]:
#Make a running bar chart for states with Covid related cases
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.animation as animation
import matplotlib.colors as mc
import colorsys
import datetime
from random import randint
import re

In [2]:
df = pd.read_csv("us-counties.csv")

In [3]:
df = df[['state', 'date', 'cases']]
df['date'] = pd.to_datetime(df.date)
df = pd.DataFrame(df)

In [6]:
df = df.groupby(['state','date'], as_index=False)['cases'].sum()
df = df.sort_values(['date'], ascending=False)
df

,state,date,cases
2048,Wyoming,2020-04-08,200
1717,Texas,2020-04-08,9900
1021,Missouri,2020-04-08,3325
1461,Oregon,2020-04-08,1239
1660,Tennessee,2020-04-08,4246
2019,Wisconsin,2020-04-08,2756
1853,Virginia,2020-04-08,3644
1048,Montana,2020-04-08,332
1420,Oklahoma,2020-04-08,1524
524,Idaho,2020-04-08,1232
